In [1]:
import glob
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
fname = 'test/vid_1_short.mp4'

In [3]:
print('Video file to be loaded: ' + str(fname))
cap = cv2.VideoCapture(str(fname))
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

Video file to be loaded: test/vid_1_short.mp4


In [4]:
os.system('rm -r runs')

0

In [6]:
os.system(f'/home/salen/miniconda3/envs/torch/bin/python detect.py --source {fname} --half --img 800 --conf-thres 0.6 --iou-thres 0.01 --weights stage_0_weights/full.pt --name "video_test" --save-txt --save-conf --nosave')

detect: weights=['stage_0_weights/full.pt'], source=test/vid_1_short.mp4, data=data/coco128.yaml, imgsz=[800, 800], conf_thres=0.6, iou_thres=0.01, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=video_test, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=True, dnn=False, vid_stride=1
YOLOv5 🚀 54eab55 Python-3.11.7 torch-2.1.2 CUDA:0 (NVIDIA GeForce RTX 4090, 24184MiB)

Fusing layers... 
Model summary: 416 layers, 140086216 parameters, 0 gradients, 208.1 GFLOPs
WARNING ⚠️ --img-size [800, 800] must be multiple of max stride 64, updating to [832, 832]
video 1/1 (1/62) /home/salen/surge/docker_sub/surgvu2024-category1/test/vid_1_short.mp4: 704x832 1 monopolar_curved_scissors, 88.2ms
video 1/1 (2/62) /home/salen/surge/docker_sub/surgvu2024-category1/test/vid_1_short.mp4: 704x8

requirements: /home/salen/surge/docker_sub/surgvu2024-category1/requirements.txt not found, check failed.


0

In [7]:
all_dets = glob.glob('runs/detect/video_test/labels/*.txt')

In [8]:
all_index = [int(i.split('/')[-1].split('.')[0].split('_')[-1])-1 for i in all_dets]

In [9]:
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

In [10]:
num_to_lab = {
            0: 'bipolar_forceps', 
            1: 'needle_driver', 
            2: 'cadiere_forceps', 
            3: 'monopolar_curved_scissor', 
            4: 'vessel_sealer', 
            5: 'force_bipolar', 
            6: 'prograsp_forceps', 
            7: 'permanent_cautery_hook_spatula', 
            8: 'stapler', 
            9: 'grasping_retractor', 
            10: 'clip_applier', 
            11: 'tip_up_fenestrated_grasper', 
            12: 'suction_irrigator', 
            }

In [11]:
def generate_bbox(frame_id, width, height):
    # bbox coordinates are the four corners of a box: [x, y, 0.5]
    # Starting with top left as first corner, then following the clockwise sequence
    # origin is defined as the top left corner of the video frame

    if frame_id in all_index:
        det_file = all_dets[all_index.index(frame_id)]
    with open(det_file, 'r') as f:
        lines = f.readlines()
        f.close()
        
    predictions = []
    for line in lines:

        cls, x_c, y_c, w, h, prob = map(float, line.split())
        
        name = f'slice_nr_{frame_id}_' + num_to_lab[int(cls)]
        bbox = [[(x_c-w/2)*width, (y_c-h/2)*height, 0.5],
                [(x_c+w/2)*width, (y_c-h/2)*height, 0.5],
                [(x_c+w/2)*width, (y_c+h/2)*height, 0.5],
                [(x_c-w/2)*width, (y_c+h/2)*height, 0.5]]
        
        prediction = {"corners": bbox, "name": name, "probability": prob}
        predictions.append(prediction)
        
    return predictions

In [12]:
generate_bbox(0, width, height)

[{'corners': [[360.0, 174.00012800000002, 0.5],
   [440.0, 174.00012800000002, 0.5],
   [440.0, 227.00032, 0.5],
   [360.0, 227.00032, 0.5]],
  'name': 'slice_nr_0_monopolar_curved_scissor',
  'probability': 0.915039}]

In [40]:
ret, frame = cap.read()

In [42]:
img = frame.copy()

In [22]:
all_index

[18,
 26,
 2,
 22,
 49,
 19,
 37,
 27,
 0,
 45,
 41,
 42,
 52,
 29,
 20,
 9,
 12,
 8,
 38,
 21,
 48,
 39,
 58,
 10,
 5,
 53,
 23,
 35,
 6,
 13,
 47,
 31,
 36,
 59,
 57,
 55,
 44,
 56,
 25,
 50,
 14,
 30,
 4,
 16,
 24,
 17,
 11,
 51,
 1,
 7,
 34,
 3,
 40,
 43,
 15,
 33,
 46,
 32,
 28,
 54]

In [23]:
num_frames

62

In [9]:
all_frames_predicted_outputs = []

for frame_id in range(50*60*100):

    name = f'slice_nr_{frame_id}_monopolar_curved_scissor'
    bbox = [[100, 100, 0.5],
            [100, 100, 0.5],
            [100, 100, 0.5],
            [100, 100, 0.5]]
    
    prediction = {"corners": bbox, "name": name, "probability": 0.99989898989}

    all_frames_predicted_outputs.append(prediction)

In [10]:
len(all_frames_predicted_outputs)

300000

In [11]:
import json

In [12]:
with open('save_test.json', 'w') as f:
    json.dump(all_frames_predicted_outputs, f)
    f.close()

In [22]:
os.system('which python')

256

In [13]:
import subprocess
import sys

In [17]:
subprocess.Popen([sys.executable, 'detect.py --source {file_path} --half --img 800 --conf-thres 0.6 --iou-thres 0.01 --weights stage_0_weights/full.pt --name "video_test" --save-txt --save-conf --nosave'],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)

<Popen: returncode: None args: ['/home/salen/miniconda3/envs/torch/bin/pytho...>

In [ ]:
, '--source {file_path} --half --img 800 --conf-thres 0.6 --iou-thres 0.01 --weights stage_0_weights/full.pt --name "video_test" --save-txt --save-conf --nosave'

In [23]:
subprocess.run('which python', shell=True, env=os.environ.copy())

CompletedProcess(args='which python', returncode=1)

In [20]:
subprocess.run(['/home/salen/miniconda3/envs/torch/bin/python', 'detect.py'], shell=True)

CompletedProcess(args=['/home/salen/miniconda3/envs/torch/bin/python', 'detect.py'], returncode=0)

In [8]:
os.environ['PATH']

'/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin'